In [1]:
from typing import List

cups_str = "712643589"

def get_cups_list() -> List[int]:
    return list(map(int, list(cups_str)))

The cup game requires doing two things efficiently:

1. Remove and place cups in a new location (moving the 3 cups
next to the current cup to after the destination cup)
2. Efficiently look up the location of cups with a specific
label (to find the destination cup)

(1) can be done efficiently with a linked list and (2) can be
done efficiently with a dictionary. The `CupCircle` class
stores both a pointer to the current cup and a dictionary
that maps labels to their `CupNode` (linked list node).

Each round of the game just has to update a few pointers in
the linked list to move the 3 cups next to the current cup to
be next to the destination cup. The destination cup is found
quickly using the map from labels to nodes.

In [2]:
class CupNode:
    """Cups are stored as a circular linked list. Each cup
    has has an integer value and a pointer to the next cup
    clockwise.
    """
    def __init__(self, val: int):
        self.val = val
        self.next = None

class CupCircle:
    """Represents a current configuration of the cup circle.
    The constructor takes in a list of values and creates a
    circular linked list. The current cup is initialized to
    the first cup in the list.
    """
    def __init__(self, cup_vals: List[int]):
        self._min_val = float("inf")
        self._max_val = float("-inf")

        self._cur_cup = CupNode(cup_vals[0])
        prev = self._cur_cup

        self._cup_map = {
            cup_vals[0]: self._cur_cup
        }

        for cup_val in cup_vals[1:]:
            self._min_val = min(self._min_val, cup_val)
            self._max_val = max(self._max_val, cup_val)
            prev.next = CupNode(cup_val)
            prev = prev.next
            self._cup_map[cup_val] = prev

        prev.next = self._cur_cup

    def get_cup(self, cup_val: int) -> CupNode:
        """Given a cups label, return corresponding CupNode"""
        return self._cup_map[cup_val]

    def _remove_3_cups(self) -> List[CupNode]:
        """Removes 3 cups next to current cup and returns a
        list with the removed cups.
        """
        removed_cups = [
            self._cur_cup.next,
            self._cur_cup.next.next,
            self._cur_cup.next.next.next,
        ]
        self._cur_cup.next = self._cur_cup.next.next.next.next
        return removed_cups

    def _get_destination_cup(self, removed_cups: List[CupNode]) -> CupNode:
        """Gets the destination cup which is defined as the cup
        with the largest label less than the current label and
        not in the removed cups list. If there are no labels
        less than the current label, then the destination cup
        is the largest label.
        """
        removed_vals = set(cup.val for cup in removed_cups)
        dest_val = self._cur_cup.val - 1

        while True:
            if dest_val >= self._min_val and dest_val not in removed_vals:
                return self.get_cup(dest_val)
            elif dest_val < self._min_val:
                dest_val = self._max_val
            else:
                dest_val -= 1

    def _place_after_dest_cup(self, dest_cup: CupNode, removed_cups: List[CupNode]):
        """Places the removed cups after the destination cup."""
        for removed_cup in reversed(removed_cups):
            removed_cup.next = dest_cup.next
            dest_cup.next = removed_cup

    def simulate_round(self):
        """Simulates a round with the following steps
        
        1. Remove the 3 cups next to the current cup
        2. Find the destination cup (either the largest
        labeled cup less than the current cup's label, or
        the cup with the largest label)
        3. Place the removed cups next to the destination
        cup
        4. Update the current cup to be the one next to the
        current cup
        """
        removed_cups = self._remove_3_cups()
        dest_cup = self._get_destination_cup(removed_cups)
        self._place_after_dest_cup(dest_cup, removed_cups)
        self._cur_cup = self._cur_cup.next

    def simulate(self, num_rounds: int):
        """Simulate the game for given number of rounds."""
        for _ in range(num_rounds):
            self.simulate_round()

## part1

Simulate the game for 100 rounds and then find the order of
the cups clockwise from cup the with the label 1.

In [3]:
cups = get_cups_list()
cup_circle = CupCircle(cups)
cup_circle.simulate(num_rounds=100)

one_cup = cup_circle.get_cup(1)
runner = one_cup.next
cup_labels = ""

while runner is not one_cup:
    cup_labels += str(runner.val)
    runner = runner.next

cup_labels

'29385746'

## part2

Add cups up to the label one million clockwise in the circle
from the initial cup arrangement. Then simulate the game for
10 million rounds and get the product of the two cups that are
next to the cup with the label 1.

In [4]:
cups = get_cups_list()
cups.extend(
    [label for label in range(max(cups)+1, 1_000_001)]
)
cup_circle = CupCircle(cups)
cup_circle.simulate(num_rounds=10_000_000)

one_cup = cup_circle.get_cup(1)

one_cup.next.val * one_cup.next.next.val

680435423892